In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
matplotlib.rcParams['figure.figsize'] = [20, 10]

Vamos a aplicar regularización $l_2$  y regularization $l_1$ usando el siguiente ejemplo de una competición de  <kaggle.com> llamado __House Prices: Advanced Regression Techniques__, ver <https://www.kaggle.com/c/house-prices-advanced-regression-techniques>. 

Por favor baja el dataset `train.cvs`. 

In [3]:
path = "./data/train.csv"
houses = pd.read_csv(path)
houses.columns.values

FileNotFoundError: [Errno 2] File b'./data/train.csv' does not exist: b'./data/train.csv'

### Seleccionamos para evitar complejidad las columnas numericas din elementos NAs

In [ ]:
numeric_columns = houses.select_dtypes(include=[np.number]).columns
all_non_numeric_columns = houses.select_dtypes(exclude=[np.number]).columns

In [ ]:
Xy = houses[numeric_columns]
target_col = "SalePrice"

Borrando las columnas con Nas values

In [ ]:
cols_with_na = []
for col in Xy.columns.values:
    nas = sum(Xy[col].isna())
    if nas > 0:
        cols_with_na.append(col)
        print(col, sum(Xy[col].isna()))
        
Xy = Xy.drop(cols_with_na, axis=1)

Hacemos separación de datos de entrenamiento, test y validación:

In [ ]:
X = Xy.drop(target_col, axis=1)
y = Xy[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

## Método 1: Simple regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

## Método 2: Regresiónn (l2-regularización)

Dado el modeo lineal: 


$$\hat{y} = a_1x_1 + \ldots + a_nx_n + b$$



Para regularización $l_2$, la función objetivo es :

$$l(\hat{Y}) = \frac{1}{N}\sum_i^N \left(y_i - \hat{y}_i\right)^2 + \alpha \sum(a_1^2 + \ldots + a_n^2)$$

Este es llamado Ridge.




In [ ]:
from sklearn.linear_model import Ridge

for i in range(-10, 10):
    alpha = 2**i
    ridge_reg = Ridge(alpha=alpha)
    ridge_reg.fit(X_train, y_train)
    y_dev_hat = ridge_reg.predict(X_dev)
    print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)), alpha)

¿Cuál es entonces el mejor modelo?

## Lasso Regression (l1 regularización)

En este caso la función objetivo es:

$$l(\hat{Y}) = \frac{1}{N}\sum_i^N \left(y_i - \hat{y}_i\right)^2 + \alpha \sum(|a_1| + \ldots + |a_n|)$$

Este es llamado Lasso.

In [ ]:
from sklearn.linear_model import Lasso

for i in range(-10, 10):
    alpha = 2**i
    lasso_reg = Lasso(alpha=alpha)
    lasso_reg.fit(X_train, y_train)
    y_dev_hat = lasso_reg.predict(X_dev)
    print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)), alpha)

¿Cuál es el mejor modelo?


### Interpretación de los coeficientes en Lasso

En Lasso hay un interesante efecto en anular bastantes coeficientes
cuando aplicamos regularización. Veamos un ejemplo para $\alpha = 100000$

In [ ]:
lasso_reg = Lasso(alpha=100000)
lasso_reg.fit(X_train, y_train)
lasso_reg.coef_

# Elastic Net: ambas regularizaciones al mismo timepo

Hay un tercer tipo de regularización que trabaja muy bien que es la combinación de $l_2$ y $l_1$:


$$l(\hat{Y}) = \frac{1}{N}\sum_i^N \left(y_i - \hat{y}_i\right)^2 + 
\alpha_1 \sum(|a_1| + \ldots + |a_n|)+
\alpha_1 \sum((a_1)^2 + \ldots + (a_n)^2)
$$

El modelo lineal con esta función objetivo es a menudo llamdadoElastic Net. Esta tienes dos parámetros  `alpha` y `l1_ratio` y la relación entre ellos es la siguiente:

$$\alpha_1 = \texttt{alpha}\cdot\texttt{l1_ratio}$$

$$\alpha_2 = \texttt{alpha}\cdot(\texttt{1 - l1_ratio})$$

Veamos en el ejemplo anterior:

In [ ]:
from sklearn.linear_model import ElasticNet
import warnings
warnings.filterwarnings("ignore")
alpha_ratio_score = []

for i in range(-10, 10):
    for j in range(20):
        alpha = 2**i
        l1_ratio = 0.6**j
        en_reg = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
        en_reg.fit(X_train, y_train)
        y_dev_hat = en_reg.predict(X_dev)
        alpha_ratio_score.append([alpha, l1_ratio, np.sqrt(mean_squared_error(y_dev, y_dev_hat)), alpha, l1_ratio])
        
scores = pd.DataFrame({
    "alpha": [ars[0] for ars in alpha_ratio_score],
    "ratio": [ars[1] for ars in alpha_ratio_score],
    "score": [ars[2] for ars in alpha_ratio_score]
})

In [ ]:
scores.sort_values('score').head()

In [ ]:
##from sklearn.preprocessing import StandardScaler, MinMaxScaler
##scaler = StandardScaler()
#scaler = MinMaxScaler()
##reg = LinearRegression()
##scaler.fit(X_train)

##X_train_scaled = scaler.transform(X_train)
##X_dev_scaled = scaler.transform(X_dev)